In [8]:
import torch
from torch import nn
from sklearn import model_selection
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from IPython import display
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

import typing
%matplotlib inline

In [31]:
# Читаем CSV-файл
cm = pd.read_csv('nodes_parameters_300.csv')

# Выводим количество строк
print(len(cm))

# Преобразуем все объектные столбцы, за исключением "Вероятность назначения (%)"
for col in cm.select_dtypes(include=['object']).columns:
    if col != "Вероятность назначения (%)":
        cm[col] = pd.factorize(cm[col])[0]

# Обрабатываем столбец "Вероятность назначения (%)": удаляем '%' и приводим к float
if "Вероятность назначения (%)" in cm.columns:
    cm["Вероятность назначения (%)"] = cm["Вероятность назначения (%)"].str.replace('%', '', regex=False).astype(float)

# Функция для очистки данных
def clean_training_data(df):
    # 1. Удаляем дубликаты
    df = df.drop_duplicates() 

    # 2. Заполняем пропуски средним значением по числовым столбцам
    df.fillna(df.mean(numeric_only=True), inplace=True)

    # 3. Обработка числовых столбцов
    for col in df.select_dtypes(include=['number']).columns:
        df[col].fillna(df[col].mean(), inplace=True)

    # 4. Удаляем выбросы с помощью метода IQR
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

    return df

# Проверяем пропуски
missing_counts = cm.isna().sum()
print(missing_counts)

# Очищаем данные
cm = clean_training_data(cm)

# Выводим первые 2 строки
print(cm.head(2))


300
Node ID                       0
Active Conns                  0
Node Performance (Score)      0
Response Time (ms)            0
Min Data Transfer (MB)        0
Task Queue Length             0
Вероятность назначения (%)    0
dtype: int64
   Node ID  Active Conns  Node Performance (Score)  Response Time (ms)  \
0        0            63                        77                  20   
1        1            76                        84                  25   

   Min Data Transfer (MB)  Task Queue Length  Вероятность назначения (%)  
0                       8                  3                        0.37  
1                      11                  5                        0.37  


/var/folders/pr/6kk4szb92094frg09x2xvl6r0000gn/T/ipykernel_16003/1662897797.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [32]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    cm.loc[:, cm.columns != 'Вероятность назначения (%)'],
    cm['Вероятность назначения (%)'],
    test_size=0.2,
    random_state=42
    )
x_train = torch.from_numpy(x_train.values).float()
x_test = torch.from_numpy(x_test.values).float()

y_train = torch.from_numpy(y_train.values).float()
y_train = y_train.reshape(-1, 1)

y_test = torch.from_numpy(y_test.values).float()
y_test = y_test.reshape(-1, 1)

print(x_train)

x_train = cm.drop(columns=['Вероятность назначения (%)'])
y_train = cm['Вероятность назначения (%)']

x_train = torch.from_numpy(x_train.values).float()
y_train = torch.from_numpy(y_train.values).float()
y_train = y_train.reshape(-1, 1)

tensor([[232.,  67.,  89.,  28.,  16.,   7.],
        [ 59.,  77.,  87.,  27.,   9.,   1.],
        [  6., 141.,  88.,  50.,  10.,   5.],
        ...,
        [106., 186.,  75.,  46.,   6.,   5.],
        [270.,  59.,  76.,  38.,  18.,   3.],
        [102., 134.,  78.,  26.,  10.,   7.]])


In [33]:
gb_regressor = GradientBoostingRegressor(n_estimators=200, learning_rate=0.01, max_depth=10, random_state=42)
gb_regressor.fit(x_train, y_train)

# Делаем предсказание и оцениваем RMSE
preds = gb_regressor.predict(x_test)
# rmse = mean_squared_error(y_test, preds, squared=False)  # RMSE
rmse = mean_squared_error(y_test, preds) ** 0.5  # Используем корень вручную


/Users/agmac/Desktop/University/HSE/Diploma/venv/lib/python3.12/site-packages/sklearn/ensemble/_gb.py:672: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


In [34]:
print(y_test.shape)
preds = torch.tensor(preds)
print(preds.shape)

torch.Size([60, 1])
torch.Size([60])


In [35]:
# y_test = torch.tensor(y_test) if not isinstance(y_test, torch.Tensor) else y_test
# preds = torch.tensor(preds) if not isinstance(preds, torch.Tensor) else preds

# Приведение preds к [20, 1] и объединение
result = torch.cat((y_test, preds.view(-1, 1)), dim=1)

In [36]:
result

tensor([[0.2600, 0.2698],
        [0.4000, 0.3911],
        [0.4000, 0.3911],
        [0.3400, 0.3391],
        [0.3700, 0.3651],
        [0.2800, 0.2871],
        [0.3600, 0.3564],
        [0.4900, 0.4690],
        [0.2300, 0.2438],
        [0.4000, 0.3911],
        [0.4800, 0.4603],
        [0.3400, 0.3391],
        [0.3100, 0.3131],
        [0.3500, 0.3478],
        [0.3000, 0.3045],
        [0.1700, 0.1919],
        [0.4000, 0.3911],
        [0.4300, 0.4170],
        [0.2900, 0.2958],
        [0.4500, 0.4344],
        [0.2600, 0.2698],
        [0.2800, 0.2871],
        [0.3000, 0.3045],
        [0.2900, 0.2958],
        [0.3800, 0.3737],
        [0.2400, 0.2525],
        [0.4500, 0.4344],
        [0.5100, 0.4863],
        [0.2400, 0.2525],
        [0.4800, 0.4603],
        [0.2400, 0.2525],
        [0.2100, 0.2265],
        [0.2900, 0.2958],
        [0.3900, 0.3824],
        [0.5200, 0.4950],
        [0.2300, 0.2438],
        [0.4800, 0.4603],
        [0.3300, 0.3304],
        [0.3

In [37]:
percantage = abs((y_test.squeeze() - preds)/y_test.squeeze() * 100)
print(percantage)
# avg_per = torch.sum(percantage) / percantage.size(0)
# print(avg_per)

tensor([ 3.7755,  2.2352,  2.2352,  0.2653,  1.3301,  2.5488,  0.9949,  4.2855,
         6.0155,  2.2352,  4.0957,  0.2653,  1.0056,  0.6406,  1.4857, 12.8673,
         2.2352,  3.0140,  1.9989,  3.4755,  3.7755,  2.5488,  1.4857,  1.9989,
         1.6477,  5.2066,  3.4755,  4.6429,  5.2066,  4.0957,  5.2066,  7.8644,
         1.9989,  1.9490,  4.8112,  6.0155,  4.0957,  0.1326,  0.6406,  2.5075,
         0.9949,  3.1394,  0.5555,  2.5075,  4.4624,  8.9275,  6.0155,  0.6406,
         2.5488,  1.3301,  4.6429,  4.4678,  0.2653,  3.1394,  1.4857, 18.4956,
         7.8644, 23.8112,  3.1394,  2.2352], dtype=torch.float64)


In [38]:
# Фильтрация выбросов > 50%
filtered_percantage = percantage[percantage <= 50]

# Подсчет среднего без выбросов
if filtered_percantage.size(0) > 0:
    avg_per = torch.sum(filtered_percantage) / filtered_percantage.size(0)
else:
    avg_per = torch.tensor(0.0)  # Защита от деления на ноль

print("\nСреднее процентное отклонение без выбросов:")
print(avg_per)


Среднее процентное отклонение без выбросов:
tensor(3.7830, dtype=torch.float64)


Согласно полученным результатам, обученная модель может определять количество запросов в секунду с погрешнотью в 17%. Испытание было проведено на 80 входных данных для тренировки и 20 для тестирования. Использовался алгоритм обучения Gradient Boosting с фильтрацией входных параметров и преобработкой входных параметров (one-hot encoding, удаление строк со слишком большой разницей мат. ожидания)